In [ ]:
import random
import numpy as np
import math
import matplotlib.pyplot as plt

NUM_CUSTOMERS = 10
NUM_VEHICLES = 3
DEPOT = (50, 50)

customers = [(58, 93), (1, 19), (92, 30), (73, 89), (59, 14), (28, 8), (26, 13), (31, 24), (86, 60), (11, 34)]

def create_individual():
    return random.sample(range(1, len(customers) + 1), len(customers))

locations = [DEPOT] + customers
print(locations)

In [ ]:
def fitness(individual):
    total_distance = 0
    prev = DEPOT

    for idx in individual:
        curr = locations[idx]
        distance = math.hypot(curr[0] - prev[0], curr[1] - prev[1])
        total_distance += distance
        prev = curr
    total_distance += math.hypot(prev[0] - DEPOT[0], prev[1] - DEPOT[1])
    return total_distance

indiv = create_individual()
print(indiv)
print(fitness(indiv))

In [ ]:
def ordered_crossover(parent1, parent2):
  start, end = sorted(random.sample(range(len(parent1)), 2))
  child = [-1]*len(parent1)
  child[start:end] = parent1[start:end]
  pointer = end
  for customer in parent2:
    if customer not in child:
      if pointer >= len(parent2):
        pointer = 0
    
      child[pointer] = customer
      pointer+=1

  return child

indiv2 = create_individual()
print(indiv)
print(indiv2)
child = ordered_crossover(indiv, indiv2)
print(child)

In [ ]:
def swap_mutation(individual, mutation_rate=0.1):
  if random.random() < mutation_rate:
    i, j = random.sample(range(len(individual)), 2)
    individual[i], individual[j] = individual[j], individual[i]
  return individual

swap_mutation(indiv)
print(indiv)

In [ ]:
def tournament_selection(pop, tournamen_size):
  tournament = random.sample(pop, tournamen_size)
  print(f"Tournament: {tournament}")
  return min(tournament, key=fitness)

list_indiv = [create_individual() for _ in range(11)]
print(tournament_selection(list_indiv, 2))

In [ ]:
def split_routes(individual, NUM_VEHICLES):
  total_stops = len(individual)
  sizes = [total_stops // NUM_VEHICLES] * NUM_VEHICLES
  for i in range(total_stops % NUM_VEHICLES):
    sizes[i] += 1
   # Sett meg i det ordentlig (Fahmi)
  routes = []
  start_index = 0

  for size in sizes:
    route = individual[start_index:start_index + size]
    print(f"Route segment ({start_index}:{start_index + size}): {route}")
    routes.append(route)
    start_index += size

  return routes


print(split_routes(indiv, NUM_VEHICLES))

In [ ]:
def genetic_algorithm(population_size=10, generations=3, mutation_rate=0.1, tournament_size=3):
  population = [create_individual() for _ in range(population_size)]
  print(f"Population: {population}")

  for generation in range(generations):
    print(f"\nGeneration: {generation}")

    for i, ind in enumerate(population):
      print(f"Individual: {ind}")
      print(f"Fitness: {fitness(ind)}")

    fitnesses = [fitness(individual) for individual in population]
    #print(f"Fitnesses: {fitnesses}")
    new_population = []
    best_idx = int(np.argmin(fitnesses))
    print(f"Index of best individual of generation {generation}: {best_idx}")
    best_individual = population[best_idx]
    new_population.append(best_individual)

    for i in range(0, NUM_CUSTOMERS):
      parent1 = tournament_selection(population, tournament_size)
      parent2 = tournament_selection(population, tournament_size)
      child1 = ordered_crossover(parent1, parent2)
      child2 = ordered_crossover(parent2, parent1)

      child1 = swap_mutation(child1, mutation_rate)
      child2 = swap_mutation(child2, mutation_rate)

      new_population.extend([child1, child2])

    population = new_population

  best_individual = min(population, key=fitness)
  print(f"Best individual: {best_individual}")
  print(f"Fitness of best individual: {fitness(best_individual)}")
  return best_individual




In [ ]:
best_sol = genetic_algorithm(generations=300, tournament_size=4)

# Sjekk at best_sol inneholder alle kunder
assert set(best_sol) == set(range(1, len(customers) + 1)), "Best solution mangler noen kunder!"

routes = split_routes(best_sol, NUM_VEHICLES)

# Sjekk at split_routes har dekket alle kundene
all_customers_in_routes = [customer for route in routes for customer in route]
assert set(all_customers_in_routes) == set(best_sol), "Noen kunder hoppet over i split_routes!"

print('Routes (customers indices):')
for i, r in enumerate(routes):
    print(f' Vehicle {i+1}:', r)

# Plot routes on plane
plt.figure(figsize=(6,6))
plt.scatter([p[0] for p in customers], [p[1] for p in customers], label='Customers')
plt.scatter(DEPOT[0], DEPOT[1], marker='s', s=80, label='Depot')
for i, route in enumerate(routes):
    xs = [DEPOT[0]] + [locations[c][0] for c in route] + [DEPOT[0]]
    ys = [DEPOT[1]] + [locations[c][1] for c in route] + [DEPOT[1]]
    plt.plot(xs, ys, label=f'Vehicle {i+1}')
plt.title('Best VRP routes (unconstrained split)')
plt.legend()
plt.show()


In [ ]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt

NUM_GENERATIONS = 500
MUTATION_PROB = 0.1

#DEPOT = (random.randint(25, 75), random.randint(25, 75))
DEPOT = (50, 50)

X_RANGE = (0, 100)
Y_RANGE = (0, 100)

# --- Helper Functions ---
def generate_customers(population_size, x_range, y_range):
    """Generate random customer coordinates within given ranges."""
    return [(random.randint(*x_range), random.randint(*y_range)) for _ in range (population_size)]

def euclidean_distance(p1, p2):
    """Calculate Euclidean distance between two points."""
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])


def calculate_distance_matrix_simple(locations):
    """Precompute pairwise distances between all locations."""
    n = len(locations)
    return [[euclidean_distance(locations[i], locations[j]) for j in range(n)] for i in range(n)]



def calculate_distance_matrix(locations):
    """Precompute pairwise distances between all locations."""
    N = len(locations)
    distance_matrix = [[0.0 for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            distance_matrix[i][j] = euclidean_distance(locations[i], locations[j])
    # Optional: Pretty print
    #print("\nDistance Matrix:")
    #for row in distance_matrix:
    #    print(row)
    return distance_matrix



# --- Genetic Algorithm Components ---
def create_individual(num_customers, num_vehicles):
    """Randomly assign customers to vehicle routes, each starting and ending at depot (0)."""
    customers = list(range(1, num_customers + 1))
    random.shuffle(customers)

    routes = [[] for _ in range(num_vehicles)]
    for i, customer_index in enumerate(customers):
        routes[i % num_vehicles].append(customer_index)

    return [[0] + route + [0] for route in routes]
    vehicle_routes = []
    for route in split_routes:
        #print(f"Route:", route)
        vehicle_routes.append([0] + route + [0])
    #return random.shuffle(list(range(1, POPULATION_SIZE + 1)))
    return vehicle_routes




def solution_distance(solution, distance_matrix):
    """Compute total distance of all routes in a solution."""
    total = 0.0
    for route in solution:
        for i in range(len(route) - 1):
            total += distance_matrix[route[i]][route[i+1]]
    
    return total


def flatten_routes(solution):
    """Flatten vehicle routes into a single customer sequence (excluding depot)."""
    return [c for route in solution for c in route if c != 0]
    flattened = []
    for route in individual:
        flattened += [customer for customer in route if customer != 0]
    return flattened


def rebuild_routes(flat_customers, num_vehicles):
    """Rebuild flat customer list into vehicle routes."""
    routes = [[] for _ in range(num_vehicles)]
    for i, customer in enumerate(flat_customers):
        routes[i % num_vehicles].append(customer)
    
    return [[0] + route + [0] for route in routes]



def crossover(parent1, parent2, num_vehicles):
    """Perform ordered crossover (OX) to generate a child solution."""
    
    flat1, flat2 = flatten_routes(parent1), flatten_routes(parent2)

    # Perform ordered crossover (OX)
    start, end = sorted(random.sample(range(len(flat1)), 2))
    child_flat = [-1] * len(flat1)
    child_flat[start:end] = flat1[start:end]

    pointer = end
    for customer in flat2:
        if customer not in child_flat:
            if pointer >= len(flat1):
                pointer = 0
            child_flat[pointer] = customer
            pointer += 1

    return rebuild_routes(child_flat, num_vehicles)



def tournament_selection(population, distance_matrix, k=3):
    """Select one solution via tournament selection."""
    candidates = random.sample(population, k)
    #print(f"Tournament: {tournament}")
    return min(candidates, key=lambda x: solution_distance(x, distance_matrix))



def mutate(solution, mutation_prob):
    """Apply swap mutation to a solution with given probability."""
    if random.random() < mutation_prob:
        for route in solution:
            if len(route) > 3: # at least 2 customers in route
                i, j = random.sample(range(1, len(route) - 1), 2)
                route[i], route[j] = route[j], route[i]
    
    return solution


# --- Main Genetic Algorithm ---
def genetic_algorithm(distance_matrix, num_customers, pop_size=20, num_vehicles=3, generations=300, mutation_prob=0.1, tournament_size=3):
    """Run a genetic algorithm for vehicle routing problem."""
    population = [create_individual(len(distance_matrix) - 1, num_vehicles) for _ in range(pop_size)]
    all_histories = []

    for gen in range(generations):
        population.sort(key=lambda s: solution_distance(s, distance_matrix))
        best_distance = solution_distance(population[0], distance_matrix)
        all_histories.append(best_distance)
        #print(f"Generation {gen}: Best Distance = {best_distance:.2f}")

        new_population = []
        while len(new_population) < pop_size:
            p1 = tournament_selection(population, distance_matrix, tournament_size)
            p2 = tournament_selection(population, distance_matrix, tournament_size)
            c1, c2 = crossover(p1, p2, num_vehicles), crossover(p2, p1, num_vehicles)

            c1 = mutate(c1, mutation_prob)
            c2 = mutate(c2, mutation_prob)
            new_population.append(c1)
            if len(new_population) < pop_size:
                new_population.append(c2)
        
        population = [population[0]] + new_population[:-1]
    
    fitnesses = [solution_distance(ind, distance_matrix) for ind in population]
    best_solution = min(population, key=lambda s: solution_distance(s, distance_matrix))
    return best_solution, solution_distance(best_solution, distance_matrix), all_histories



def run_experiment(generations, mutation_prob, distance_matrix, num_runs=20):
    results = {}
    best_routes= []
    all_histories = []
    best_solution_overall = None
    best_distance_overall = float('inf')
    for _ in range(num_runs):
        best_solution, best_distance, history = genetic_algorithm(
        distance_matrix,
        num_customers=NUM_CUSTOMERS,
        num_vehicles=NUM_VEHICLES,
        pop_size=NUM_CUSTOMERS,
        generations=NUM_GENERATIONS,
        mutation_prob=MUTATION_PROB
        )
        best_routes.append(best_distance)
        all_histories.append(history)

        if best_distance < best_distance_overall:
            best_distance_overall = best_distance
            best_solution_overall = best_solution

    avg_history = np.mean(all_histories, axis=0)
    results = {
        "best": min(best_routes),
        "worst": max(best_routes),
        "mean": np.mean(best_routes),
        "std_dev": np.std(best_routes),
        "avg_best_fitness": avg_history
    }

    print(f"Statistics for Population Size {NUM_CUSTOMERS}")
    print(f"Best: {min(best_routes):.2f}, Worst: {max(best_routes):.2f}, Mean: {np.mean(best_routes):.2f}, Std Dev: {np.std(best_routes):.2f}\n")

    return results, best_solution_overall


In [ ]:
NUM_CUSTOMERS = 25
NUM_VEHICLES = 14
# Generate customer locations
customers = generate_customers(NUM_CUSTOMERS, X_RANGE, Y_RANGE)
locations = [DEPOT] + customers

print("Locations (Customers + Depot):")
for i, loc in enumerate(locations):
    label = "Depot" if i == 0 else f"C{i}"
    print(f"{label}: {loc}")

# --- Example Individual --- 
# Create a sample individual (solution)
#individual = create_individual(POPULATION_SIZE, NUM_VEHICLES)
#print("\nSample Individual (Vehicle Routes):")
#for i, route in enumerate(individual):
#    print(f"Vehicle {i}: {route}")

distance_matrix = calculate_distance_matrix(locations)

results, best_solution = run_experiment(NUM_GENERATIONS, MUTATION_PROB, distance_matrix)

plt.figure(figsize=(12, 8))
plt.plot(results['avg_best_fitness'], linewidth=2, label="Average")
plt.xlabel("Generations")
plt.ylabel("Average Best Distance")
plt.title("Average Best Distance over Generations")
plt.legend()
plt.show()

In [ ]:
customers = [(58, 93), (1, 19), (92, 30), (73, 89), (59, 14), (28, 8), (26, 13), (31, 24), (86, 60), (11, 34)]
locations = [DEPOT] + customers
distance_matrix = calculate_distance_matrix(locations)

print("Locations (Customers + Depot):")
for i, loc in enumerate(locations):
    label = "Depot" if i == 0 else f"C{i}"
    print(f"{label}: {loc}")

NUM_CUSTOMERS = len(customers)
NUM_VEHICLES = 5

In [ ]:
results, _ = run_experiment(NUM_GENERATIONS, MUTATION_PROB, distance_matrix)

plt.figure(figsize=(12, 8))
plt.plot(results['avg_best_fitness'], linewidth=2, label="Average")
plt.xlabel("Generations")
plt.ylabel("Average Best Distance")
plt.title("Average Best Distance over Generations")
plt.legend()
plt.show()

In [ ]:
customers = [(58, 93), (1, 19), (92, 30), (73, 89), (59, 14), (28, 8), (26, 13), (31, 24), (86, 60), (11, 34), (34, 54), (2, 23), (4,34)]
NUM_CUSTOMERS = len(customers)
NUM_VEHICLES = 4
locations = [DEPOT] + customers
distance_matrix = calculate_distance_matrix(locations)

print("Locations (Customers + Depot):")
for i, loc in enumerate(locations):
    label = "Depot" if i == 0 else f"C{i}"
    print(f"{label}: {loc}")

results, best_solution = run_experiment(NUM_GENERATIONS, MUTATION_PROB, distance_matrix)
print(best_solution)

def plot_route(solution, locations):
    plt.figure(figsize=(10, 8))
    for vehicle_idx, route in enumerate(solution):
        route_coords = [locations[i] for i in route]
        xs, ys = zip(*route_coords)
        plt.plot(xs, ys, marker='o', label=f"Vehicle {vehicle_idx + 1}")

        
        print(f"Vehicle {vehicle_idx + 1}: " + " -> ".join(map(str, route)))
    
    plt.scatter(*DEPOT, color='red', s=100, label='Depot')
    for i, (x, y) in enumerate(locations[1:], start=1):
        plt.text(x, y, f"C{i}", fontsize=9)
    
    plt.title("Best Route for Vehicles")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.grid(True)
    plt.show()


plot_route(best_solution, locations)

plt.figure(figsize=(12, 8))
plt.plot(results['avg_best_fitness'], linewidth=2, label="Average")
plt.xlabel("Generations")
plt.ylabel("Average Best Distance")
plt.title("Average Best Distance over Generations")
plt.legend()
plt.show()